# Visualizing Example Cells

In [42]:
import os
from meshparty import trimesh_io, trimesh_vtk, meshwork
from caveclient import CAVEclient
import numpy as np
import pandas as pd
import cloudvolume
import itkwidgets

#setting up the cave client, if you have not done this before checkout the notebook CAVEsetup.ipynb at 
#https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/mm3_intro/CAVEsetup.ipynb
#client = CAVEclient()
#client.auth.get_new_token()
#client.auth.save_token(token="Your token here")

"""
dataset_name = 'minnie65_public_v117'
client = CAVEclient(dataset_name)
cv = cloudvolume.CloudVolume(client.info.segmentation_source(), progress=False)

#using the google cloud segmentation path
#cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)
#cv = cloudvolume.CloudVolume("precomputed://gs://allen-minnie-phase3/minniephase3-emily-pcg-skeletons/minnie_all", use_https=True)

# to enable to cache the mesh data, create a MeshMeta object:
mm = trimesh_io.MeshMeta(cv_path = client.info.segmentation_source(),
                         disk_cache_path='minnie65_v117_meshes',
                         map_gs_to_https=True)
"""


'\ndataset_name = \'minnie65_public_v117\'\nclient = CAVEclient(dataset_name)\ncv = cloudvolume.CloudVolume(client.info.segmentation_source(), progress=False)\n\n#using the google cloud segmentation path\n#cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)\n#cv = cloudvolume.CloudVolume("precomputed://gs://allen-minnie-phase3/minniephase3-emily-pcg-skeletons/minnie_all", use_https=True)\n\n# to enable to cache the mesh data, create a MeshMeta object:\nmm = trimesh_io.MeshMeta(cv_path = client.info.segmentation_source(),\n                         disk_cache_path=\'minnie65_v117_meshes\',\n                         map_gs_to_https=True)\n'

In [47]:
datafolder = os.path.join(os.path.expanduser('~'), 'work', 'pv-sst-dendrites',
                          'data', 'SchneiderMizell_et_al_2023', 'skeletons', 'meshwork')
mesh = meshwork.load_meshwork(os.path.join(datafolder, '864691137053906294.h5'))

In [54]:
mesh.vertices[mesh.anno['is_dendrite']]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [49]:
itkwidgets.view?

In [2]:
mesh = meshwork.load_meshwork('minnie65_v117_meshes/864691137053906294.h5')
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)
#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Basket Cell example

In [3]:
#selected one:
BC_ID = 864691135269913253
# see the Plot_Morphologies notebook to see them all
mesh = mm.mesh(seg_id=BC_ID) # and download it / just load it

In [8]:
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

### with synapses

In [8]:
cellid = BC_ID
mesh = cv.mesh.get(cellid,lod=3)[cellid]

post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))

poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerBC =itkwidgets.view(geometries=[poly_data],
                          point_sets = [syn_pts],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['g'])
viewerBC

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Martinotti Cell example

In [5]:
#MC_ID = 864691135467660940

In [ ]:
MC_ID = 864691137053906294 # L23 many synapses
meshw

In [6]:
#selected one:
#MC_ID = 864691137053906294 # L23 many synapses
#MC_ID = 864691135754910674 # L23 but small
MC_ID = 864691135467660940 # L5 few synapses
# see the Plot_Morphologies notebook to see them all
mesh = mm.mesh(seg_id=MC_ID) # and download it / just load it

In [7]:
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [8]:
# what MC IDs are available as meshes for display ?
MC_IDs = [864691135954232584, 864691135969342309, 864691135467660940,
          864691135737246596, 864691135575445022, 864691135988665856,
          864691135683637746, 864691135571546917, 864691135575425822,
           864691135502364341, 864691135618117263, 864691135386843521,
           864691135939865345, 864691137053906294, 864691135463913541,
           864691135416083642, 864691135800027234, 864691135754152141,
           864691136143786292, 864691135687245024, 864691135754910674,
           864691135256626095, 864691137197468481, 864691135374222153,
           864691136451445247, 864691136966045646, 864691136602069713,
           864691136109238200, 864691136065953688, 864691135851060423,
           864691135323181212, 864691135925834510, 864691135373819592,
           864691135404114414, 864691135660772080, 864691135699572514,
           864691135884162160, 864691135082074103, 864691135777016928,
           864691135577304325, 864691135058985115]
for i, ID in enumerate(MC_IDs):
    try:
        mesh = cv.mesh.get(ID)[ID]
        print(i+1, ID)
    except BaseException:
        pass

In [10]:
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Showing both together

In [25]:
cellid = MC_ID
mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data1 = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)
cellid = BC_ID
mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data2 = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data1, poly_data2])

In [26]:
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Basket cell with synapses

In [6]:
cellid = BC_ID

mesh = cv.mesh.get(cellid,lod=3)[cellid]

post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))

poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerBC =itkwidgets.view(geometries=[poly_data],
                          point_sets = [syn_pts],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['r'])
viewerBC

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [27]:
cellid = MC_ID

mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerMC =itkwidgets.view(geometries=[poly_data],
                          point_sets = [syn_pts],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['r'])
viewerMC

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## More controlled rendering with `trimesh_vtk.render_actor`

In [ ]:
cellid = MC_ID

# mesh
mesh = cv.mesh.get(cellid,lod=3)[cellid]
mesh_actors = trimesh_vtk.mesh_actor(mesh, color=(0, 0, 0.4))
voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm

# synapses
post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
syn_actors = trimesh_vtk.point_cloud_actor(syn_pts, size=syn_sizes/20, color=(1, 0, 0))
#                                           color=[() for i in range(len(syn_pts))])

#camera = trimesh_vtk.camera_from_quat()
trimesh_vtk.render_actors([syn_actors, mesh_actors])

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
ERROR:root:Cannot create GLX context.  Aborting.


In [69]:
trimesh_vtk.camera_from_quat?